In [1]:
# Import libraries
import os
import pandas as pd
import sqlite3

In [2]:
# Set up environment
try:
    path_sources = os.chdir("../Sources") # Back one folder in current directory
except:
    print("Path does not exists")

In [3]:
db_etl = 'ADV_MX.db'
target_table = 'SALES2013AND2014'
xlsx_name = 'SalesFile.xlsx'
usr = 'usrhs'

In [4]:
# Enable connection

def sqlite_connection(db_file):
    try:
        connection_path = sqlite3.connect(db_file)
        cursor_obj = connection_path.cursor()
        print(f"SUCCESS CONNECTION\nVersion SQlite: {sqlite3.version}")
        return connection_path,cursor_obj
    except sqlite3.Error as e:
        print(f"FAILED CONNECTION\nERROR is: {e}")

In [5]:
con,cur = sqlite_connection(db_etl)

SUCCESS CONNECTION
Version SQlite: 2.6.0


In [6]:
def sqlite_query(connection_path,query_str):
    try:
        connection_path.execute(query_str)
        connection_path.commit()
        print("SUCCESS EXECUTION")
    except Exception as e:
        print("FAILED EXECUTION\n")
        print(e)

In [7]:
# File to load
df_sales = pd.read_excel(xlsx_name)

In [8]:
df_sales.shape[0] # Total Records

51

In [9]:
df_sales # FIRST RECORDS

,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Sales,Profit,Date,Month Number
0,TrailBlazer 1000,non value,958,5.0,300.0,287400.00,47900.00,2014-08-01,8.0
1,TrailBlazer 2000,Low,2529,10.0,7.0,17525.97,4880.97,2014-07-01,7.0
2,SpeedMaster 1000,Low,918,10.0,300.0,269892.00,40392.00,2014-05-01,5.0
3,SpeedMaster 2000,Low,1774,10.0,125.0,215097.50,2217.50,2014-03-01,3.0
4,Explorer 1000,Low,866,250.0,NaN,9976.32,7378.32,2014-05-01,5.0
5,Explorer 2000,Medium,NaN,NaN,15.0,7908.75,2358.75,2014-01-01,1.0
6,GravityMaster 1000,Medium,1679,260.0,350.0,552391.00,115851.00,2014-09-01,9.0
7,GravityMaster 2000,Medium,588,120.0,20.0,10936.80,5056.80,2013-12-01,12.0
8,Pathfinder 1000,Medium,1366,260.0,20.0,25134.40,11474.40,2014-06-01,6.0
9,Pathfinder 2000,Medium,973,10.0,20.0,NaN,7978.60,2014-03-01,3.0


In [10]:
# Define raw table extructure with guidelines

# Drop raw table
sqlite_query(con,f"""DROP TABLE IF EXISTS {target_table + '_RAW'};""")

sales_raw = f"""
            CREATE TABLE {target_table + '_RAW'} (
            "PRODUCT_NAME" TEXT,
            "DISCOUNT_BAND" TEXT,
            "UNITS_SOLD" TEXT,
            "MANUFACTURING_PRICE" TEXT,
            "SALE_PRICE" TEXT,
            "SALES" TEXT,
            "PROFIT" TEXT,
            "DATE" TEXT,
            "MONTH_NUMBER" TEXT);"""

sqlite_query(con,sales_raw)

SUCCESS EXECUTION
SUCCESS EXECUTION


In [11]:
# Insert raw data into raw table
def sqlite_insert(connection_path,query_insert,df_pandas):
    try:
        print(f'Total records to insert are: {df_pandas.shape[0]} and the total columns are: {df_pandas.shape[1]}')
        data_to_add = [tuple(r) for r in df_pandas.to_numpy()]
        connection_path.executemany(query_insert,data_to_add)
        connection_path.commit()
        print('SUCCESS INSERT')
    except Exception as e:
        print("Failed execution\n")
        print(e)

In [12]:
insert_raw = f"""INSERT INTO {target_table + '_RAW'}
                ("PRODUCT_NAME","DISCOUNT_BAND","UNITS_SOLD","MANUFACTURING_PRICE","SALE_PRICE","SALES","PROFIT","DATE","MONTH_NUMBER")
                VALUES (?,?,?,?,?,?,?,?,?)"""

sqlite_insert(con,insert_raw,df_sales)

Total records to insert are: 51 and the total columns are: 9
SUCCESS INSERT


In [13]:
# Check values and see the issues in the different columns

df_raw = pd.read_sql_query(f"""SELECT * FROM {target_table + '_RAW'}""", con)

print(f"Total records are: {df_raw.shape[0]}")
display(df_raw)

Total records are: 51


,PRODUCT_NAME,DISCOUNT_BAND,UNITS_SOLD,MANUFACTURING_PRICE,SALE_PRICE,SALES,PROFIT,DATE,MONTH_NUMBER
0,TrailBlazer 1000,non value,958,5.0,300.0,287400.0,47900.0,2014-08-01,8.0
1,TrailBlazer 2000,Low,2529,10.0,7.0,17525.97,4880.97,2014-07-01,7.0
2,SpeedMaster 1000,Low,918,10.0,300.0,269892.0,40392.0,2014-05-01,5.0
3,SpeedMaster 2000,Low,1774,10.0,125.0,215097.5,2217.5,2014-03-01,3.0
4,Explorer 1000,Low,866,250.0,None,9976.32,7378.32,2014-05-01,5.0
5,Explorer 2000,Medium,None,None,15.0,7908.75,2358.75,2014-01-01,1.0
6,GravityMaster 1000,Medium,1679,260.0,350.0,552391.0,115851.0,2014-09-01,9.0
7,GravityMaster 2000,Medium,588,120.0,20.0,10936.8,5056.8,2013-12-01,12.0
8,Pathfinder 1000,Medium,1366,260.0,20.0,25134.4,11474.4,2014-06-01,6.0
9,Pathfinder 2000,Medium,973,10.0,20.0,None,7978.6,2014-03-01,3.0


In [14]:
""" Data cleaning using stage tables
Let see some issues in this dataset
1. Notices UNITS_SOLD, SALE_PRICE, SALES and PROFIT has null values 
2. Notices UNITS_SOLD has text quantities
3. Notices DISCOUNT_BAND has a value not requiered
4. Notices DATE has null values, business say this data correspond to date 03 March 2023
5. Notices business does not requiere MANUFACTURING_PRICE null
6. Notices in this records are duplicated values
7. Add control columns
"""

insert_stg = f"""SELECT DISTINCT
                PRODUCT_NAME,
                CASE WHEN DISCOUNT_BAND = 'non value' THEN NULL ELSE DISCOUNT_BAND END AS DISCOUNT_BAND,
                COALESCE(CASE WHEN UNITS_SOLD = 'six hundred' THEN 600 ELSE UNITS_SOLD END,0) AS UNITS_SOLD,
                MANUFACTURING_PRICE,
                COALESCE(SALE_PRICE,0) AS SALE_PRICE,
                COALESCE(SALES,0) AS SALES,
                COALESCE(PROFIT,0) AS PROFIT,
                COALESCE(DATE,'2023-03-03') AS DATE,
                STRFTIME('%m',COALESCE(DATE,'2023-03-03')) AS MONTH_NUMBER,
                '{usr}' AS USR_LOAD,
                DATE('now') AS CREATE_DT
                FROM {target_table+ '_RAW'}
                WHERE MANUFACTURING_PRICE IS NOT NULL; """

In [15]:
# Define target table extructure with guidelines

sales_tgt = f"""
            CREATE TABLE IF NOT EXISTS {target_table} (
            "PRODUCT_NAME" TEXT,
            "DISCOUNT_BAND" TEXT,
            "UNITS_SOLD" TEXT,
            "MANUFACTURING_PRICE" TEXT,
            "SALE_PRICE" REAL,
            "SALES" REAL,
            "PROFIT" REAL,
            "DATE" TEXT,
            "MONTH_NUMBER" INTEGER,
            "USR_LOAD" TEXT,
            "CREATE_DT" TEXT);"""

sqlite_query(con,sales_tgt)

insert_tgt = f"""INSERT INTO {target_table}
            {insert_stg}"""

sqlite_query(con,insert_tgt)

SUCCESS EXECUTION
SUCCESS EXECUTION


In [16]:
df_describe = pd.read_sql_query(f"""PRAGMA table_info({target_table});""", con)

display(df_describe)

,cid,name,type,notnull,dflt_value,pk
0,0,PRODUCT_NAME,TEXT,0,None,0
1,1,DISCOUNT_BAND,TEXT,0,None,0
2,2,UNITS_SOLD,TEXT,0,None,0
3,3,MANUFACTURING_PRICE,TEXT,0,None,0
4,4,SALE_PRICE,REAL,0,None,0
5,5,SALES,REAL,0,None,0
6,6,PROFIT,REAL,0,None,0
7,7,DATE,TEXT,0,None,0
8,8,MONTH_NUMBER,INTEGER,0,None,0
9,9,USR_LOAD,TEXT,0,None,0


In [17]:
# Target table

display(pd.read_sql_query(f"""SELECT COUNT(*) FROM {target_table}""", con))

,COUNT(*)
0,41


In [18]:
display(pd.read_sql_query(f"""SELECT * FROM {target_table}""", con))

,PRODUCT_NAME,DISCOUNT_BAND,UNITS_SOLD,MANUFACTURING_PRICE,SALE_PRICE,SALES,PROFIT,DATE,MONTH_NUMBER,USR_LOAD,CREATE_DT
0,TrailBlazer 1000,None,958,5.0,300.0,287400.00,47900.00,2014-08-01,8,usrhs,2023-11-17
1,TrailBlazer 2000,Low,2529,10.0,7.0,17525.97,4880.97,2014-07-01,7,usrhs,2023-11-17
2,SpeedMaster 1000,Low,918,10.0,300.0,269892.00,40392.00,2014-05-01,5,usrhs,2023-11-17
3,SpeedMaster 2000,Low,1774,10.0,125.0,215097.50,2217.50,2014-03-01,3,usrhs,2023-11-17
4,Explorer 1000,Low,866,250.0,0.0,9976.32,7378.32,2014-05-01,5,usrhs,2023-11-17
5,GravityMaster 1000,Medium,1679,260.0,350.0,552391.00,115851.00,2014-09-01,9,usrhs,2023-11-17
6,GravityMaster 2000,Medium,588,120.0,20.0,10936.80,5056.80,2013-12-01,12,usrhs,2023-11-17
7,Pathfinder 1000,Medium,1366,260.0,20.0,25134.40,11474.40,2014-06-01,6,usrhs,2023-11-17
8,Pathfinder 2000,Medium,973,10.0,20.0,0.00,7978.60,2014-03-01,3,usrhs,2023-11-17
9,Voyager 1000,High,2072,260.0,15.0,27972.00,7252.00,2014-12-01,12,usrhs,2023-11-17


In [19]:
# Close connection

con.close()